# RunnableParallel

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [1]:
pip show langchain

Name: langchain
Version: 1.0.7
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /opt/anaconda3/envs/LangChain_course_env1/lib/python3.13/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append("..")
import sks_config

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [4]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

In [5]:
chat = ChatOpenAI(api_key = sks_config.SKS_OPENAI_API_KEY,
                  model_name = 'gpt-4o-mini', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

/opt/anaconda3/envs/LangChain_course_env1/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3639: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [6]:
string_parser = StrOutputParser()

In [7]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [8]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [9]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1. "Fluent Python" by Luciano Ramalho  \n2. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin  \n3. "Python Cookbook" by David Beazley and Brian K. Jones  ',
 'projects': '1. Personal Finance Tracker with Data Visualization  \n2. Web Scraper with Automated Reporting  \n3. Chatbot using Natural Language Processing'}

In [10]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [11]:
%%time
chain_books.invoke({'programming language':'Python'})

CPU times: user 9.04 ms, sys: 3.72 ms, total: 12.8 ms
Wall time: 1.86 s


'1. "Fluent Python" by Luciano Ramalho  \n2. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin  \n3. "Python Cookbook" by David Beazley and Brian K. Jones  '

In [12]:
%%time
chain_projects.invoke({'programming language':'Python'})

CPU times: user 8.24 ms, sys: 2.22 ms, total: 10.5 ms
Wall time: 1.15 s


'1. Personal Finance Tracker\n2. Web Scraper with Data Visualization\n3. Chatbot using Natural Language Processing'

In [13]:
%%time
chain_parallel.invoke({'programming language':'Python'})

CPU times: user 21.8 ms, sys: 4.4 ms, total: 26.2 ms
Wall time: 2.76 s


{'books': '1. "Fluent Python" by Luciano Ramalho  \n2. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin  \n3. "Python Cookbook" by David Beazley and Brian K. Jones  ',
 'projects': '1. Personal Finance Tracker\n2. Web Scraper with Data Visualization\n3. Chatbot using Natural Language Processing'}